In [172]:
import os
os.environ['USE_PYGEOS'] = '0'
import scripts.utils as utils
import pandas as pd

config = utils.load_config("scripts")
tempdir = config['paths']['tempdir']
datadir = config['paths']['datadir']
outdir = config['paths']['tempdir']
figdir = config['paths']['figdir']

SCENARIO = "nf"

# Time series for each event
## Create dataframe of #LoS days per month
Create a time series of the number of restriction events per month for each of L1 - L4 events from Anna's raw WREW data. To have a time series of >L2 events just sum L2 + L3 + L4.

In [173]:
# want it to look like this:
df_target = pd.read_csv(os.path.join(datadir, 'los', f'weighted_monthly_LoS_{SCENARIO}.csv'), header=1)
df_target.head()

/var/folders/7w/k72v6f5x25gblsqmh6bxd3j40000gn/T/ipykernel_24283/980646105.py:2: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_target = pd.read_csv(os.path.join(datadir, 'los', f'weighted_monthly_LoS_{SCENARIO}.csv'), header=1)


,Node,Unnamed: 1,Unnamed: 2,Affinity WRZ 124 (LoS),Affinity WRZ 35 (LoS),Bristol WRZ (LoS),Cambs & West Suffolk (Los),ESW Essex WRZ (LoS),East Colliford WRZ (LoS),East Roadford WRZ (LoS),...,Teeside WRZ (LoS),Tyneside WRZ (LoS),Tywi CUS (LoS),West Colliford WRZ (LoS),West Cumbria (LoS),Wimbleball WRZ (LoS),YW Central (LoS),YW NW (LoS),YW S (LoS),YW SW (LoS)
0,Ensemble,Year,Month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,2020,1,30.0,30.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2020,2,28.0,28.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,2020,3,31.0,31.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,2020,4,30.0,30.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [174]:
dfs_binary = []
dfs_l1 = []
dfs_l2 = []
dfs_l3 = []
dfs_l4 = []


for ensemble in range(99):
    df = pd.read_csv(os.path.join(datadir, 'los', f'Phase_3_{SCENARIO.upper()}', f'jobid_{ensemble}_National_Model_globalPlotVars_selected.csv'), header=1)

    los_cols = [col for col in df.columns if "LoS" in col]
    df = df[['Year', 'Day'] + los_cols]
    df['Date'] = pd.to_datetime(df['Year'] * 1000 + df['Day'], format='%Y%j')
    df = df.drop(columns=["Day", "Year"])
    df = df.set_index("Date")

    df_binary = (df > 0).astype(int).groupby(pd.Grouper(freq="M")).agg(sum)
    df_l1 = (df == 1).astype(int).groupby(pd.Grouper(freq="M")).agg(sum)
    df_l2 = (df == 2).astype(int).groupby(pd.Grouper(freq="M")).agg(sum)
    df_l3 = (df == 3).astype(int).groupby(pd.Grouper(freq="M")).agg(sum)
    df_l4 = (df == 4).astype(int).groupby(pd.Grouper(freq="M")).agg(sum)
    
    df_binary['Ensemble'] = int(ensemble + 1)
    df_l1['Ensemble'] = int(ensemble + 1)
    df_l2['Ensemble'] = int(ensemble + 1)
    df_l3['Ensemble'] = int(ensemble + 1)
    df_l4['Ensemble'] = int(ensemble + 1)
 
    dfs_binary.append(df_binary)
    dfs_l1.append(df_l1)
    dfs_l2.append(df_l2)
    dfs_l3.append(df_l3)
    dfs_l4.append(df_l4)
    
    
df_binary = pd.concat(dfs_binary, axis=0).sort_values(by=["Date", "Ensemble"])
df_l1 = pd.concat(dfs_l1, axis=0).sort_values(by=["Date", "Ensemble"])
df_l2 = pd.concat(dfs_l2, axis=0).sort_values(by=["Date", "Ensemble"])
df_l3 = pd.concat(dfs_l3, axis=0).sort_values(by=["Date", "Ensemble"])
df_l4 = pd.concat(dfs_l4, axis=0).sort_values(by=["Date", "Ensemble"])

df_binary.head()

,STW (LoS),Sunderland WRZ (LoS),Tyneside WRZ (LoS),Teeside WRZ (LoS),Bristol WRZ (LoS),Parrett Demands (LoS),East Colliford WRZ (LoS),West Colliford WRZ (LoS),North Roadford WRZ (LoS),Roadford WRZ (LoS),...,Tywi CUS (LoS),SWS South Hants (LoS),PRT (LoS),SWS South Hants (LoS) updated,West Cumbria (LoS),Fenland (LoS),Lincolnshire (LoS),Affinity WRZ 124 (LoS),Affinity WRZ 35 (LoS),Ensemble
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,30,30,1
2020-01-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,30,2
2020-01-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,30,3
2020-01-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,30,4
2020-01-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,30,5


In [175]:
old_cols = [*df_target.columns]
new_cols = [*df_binary.columns]

diff1 = set(new_cols) - set(old_cols)
diff2 = set(old_cols) - set(new_cols)

print("Elements in new but not in old:", diff1)
print("Elements in old but not in new:", diff2)

Elements in new but not in old: {'STW (LoS)', 'SWS South Hants (LoS)', 'Ensemble'}
Elements in old but not in new: {'Unnamed: 1', 'Node', 'Cambs & West Suffolk (Los)', 'Unnamed: 2'}


## 2. Melted version

In [176]:
# want it to look like this:
ls_df = pd.read_csv(os.path.join(tempdir, SCENARIO, "monthly_los_melted.csv"), index_col=[0])
ls_df.head()

,RZ_ID,Year,Month,ensemble,LoS
0,1,2020,2,NF1,28.0
1,1,2020,3,NF1,31.0
2,1,2020,4,NF1,30.0
3,1,2020,5,NF1,4.0
4,1,2020,6,NF1,0.0


In [177]:
# scripts.from_process_los_data.py
wrz_code = pd.read_excel(os.path.join(datadir, 'WRZ', 'wrz_code.xlsx'))
wrz_code['RZ ID'] = pd.to_numeric(wrz_code['RZ ID'])
wrz_dict = {wrz_code: f'{rz_id:.0f}' for wrz_code, rz_id in zip(wrz_code['WREW Code'], wrz_code['RZ ID'])}

In [191]:
def melt_los_df(df):
    rz_id_cols = df.columns[:-1]
    df_melted = df.reset_index()
    df_melted = df_melted.melt(id_vars=['Date', 'Ensemble'], value_vars=rz_id_cols, var_name='RZ_ID', value_name='LoS', ignore_index=True)

    df_melted = df_melted[['RZ_ID', 'Date', 'Ensemble', 'LoS']]
    df_melted['RZ_ID'] = df_melted['RZ_ID'].map(wrz_dict)
    df_melted = df_melted[df_melted['RZ_ID'].notnull()].reset_index(drop=True)
    df_melted = df_melted[df_melted['RZ_ID'] != "nan"].reset_index(drop=True)
    df_melted = df_melted.drop_duplicates()
    df_melted = df_melted.sort_values(by=['RZ_ID', "Ensemble", 'Date']).reset_index(drop=True)
    
    df_melted['Year'] = df_melted['Date'].dt.year
    df_melted['Month'] = df_melted['Date'].dt.month
    df_melted = df_melted.drop(columns='Date')
    
    df_melted = df_melted[['RZ_ID', 'Ensemble', 'Year', 'Month', 'LoS']]
    df_melted = df_melted.sort_values(by=['RZ_ID', 'Ensemble', 'Year', 'Month', 'LoS'])
    return df_melted

df_binary_melted = melt_los_df(df_binary)
df_l1_melted = melt_los_df(df_l1)
df_l2_melted = melt_los_df(df_l2)
df_l3_melted = melt_los_df(df_l3)
df_l4_melted = melt_los_df(df_l4)

df_l4_melted.head()

,RZ_ID,Ensemble,Year,Month,LoS
0,1,1,2020,1,0
1,1,1,2020,2,0
2,1,1,2020,3,0
3,1,1,2020,4,0
4,1,1,2020,5,0


In [193]:
df_binary_melted.to_csv(os.path.join(outdir, SCENARIO.lower(), 'monthly_los_level0_melted.csv'))
df_l1_melted.to_csv(os.path.join(outdir, SCENARIO.lower(), 'monthly_los_level1_melted.csv'))
df_l2_melted.to_csv(os.path.join(outdir, SCENARIO.lower(), 'monthly_los_level2_melted.csv'))
df_l3_melted.to_csv(os.path.join(outdir, SCENARIO.lower(), 'monthly_los_level3_melted.csv'))
df_l4_melted.to_csv(os.path.join(outdir, SCENARIO.lower(), 'monthly_los_level4_melted.csv'))

## 3. Convert to a time series that I can use to model

In [170]:
# or do this in the other notebook

In [171]:
tempdir

'/Users/alison/Documents/RAPID/correlation-analysis/davids_results/data_temp'